In [ ]:
import sys
!{sys.executable} -m pip install -U selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd


Requirement already up-to-date: selenium in c:\users\34690\anaconda3\lib\site-packages (3.141.0)


In [ ]:
#España
Posicion=[]
Equipo=[]
Puntos=[]
Jornada=[]
Ganados=[]
Empatados=[]
Perdidos=[]
Goles_a_favor=[]
Goles_en_contra=[]
Año=[]
Liga=[]

Equipo_Local=[]
Equipo_Visitante=[]
Resultado=[]
Año_=[]
Jornada_=[]
Fecha=[]
Liga_=[]
Ano1=1990
Ano2=2019
for dic in range(0,1):
    liga="segunda"
    valor=5
    for año in range (Ano1,Ano2):
        for jornada in range(1,44):
            driver=webdriver.Chrome('C:/Users/34690/Desktop/Imagenes presentacion/chromedriver.exe')
            driver.get("https://www.resultados-futbol.com/"+liga+str(año)+"/grupo1/jornada"+str(jornada))
            time.sleep(2)
            try:
                pulsar=driver.find_element_by_xpath("/html/body/div[1]/div/div/div/div[2]/div/button[2]")
                pulsar.click()
            except:
                a=0
            for i in range(0,25):
                try:
                    Posicion.append(driver.find_element_by_xpath("/html/body/div[4]/div[3]/div[10]/div[2]/div["+str(valor)+"]/div[5]/div[2]/div/form/table/tbody/tr["+str(i)+"]/th").text)
                    Equipo.append(driver.find_element_by_xpath("/html/body/div[4]/div[3]/div[10]/div[2]/div["+str(valor)+"]/div[5]/div[2]/div/form/table/tbody/tr["+str(i)+"]/td[1]").text)
                    Puntos.append(driver.find_element_by_xpath("/html/body/div[4]/div[3]/div[10]/div[2]/div["+str(valor)+"]/div[5]/div[2]/div/form/table/tbody/tr["+str(i)+"]/td[2]").text)
                    Jornada.append(driver.find_element_by_xpath("/html/body/div[4]/div[3]/div[10]/div[2]/div["+str(valor)+"]/div[5]/div[2]/div/form/table/tbody/tr["+str(i)+"]/td[3]").text)
                    Ganados.append(driver.find_element_by_xpath("/html/body/div[4]/div[3]/div[10]/div[2]/div["+str(valor)+"]/div[5]/div[2]/div/form/table/tbody/tr["+str(i)+"]/td[4]").text)
                    Empatados.append(driver.find_element_by_xpath("/html/body/div[4]/div[3]/div[10]/div[2]/div["+str(valor)+"]/div[5]/div[2]/div/form/table/tbody/tr["+str(i)+"]/td[5]").text)
                    Perdidos.append(driver.find_element_by_xpath("/html/body/div[4]/div[3]/div[10]/div[2]/div["+str(valor)+"]/div[5]/div[2]/div/form/table/tbody/tr["+str(i)+"]/td[6]").text)
                    Goles_a_favor.append(driver.find_element_by_xpath("/html/body/div[4]/div[3]/div[10]/div[2]/div["+str(valor)+"]/div[5]/div[2]/div/form/table/tbody/tr["+str(i)+"]/td[7]").text)
                    Goles_en_contra.append(driver.find_element_by_xpath("/html/body/div[4]/div[3]/div[10]/div[2]/div["+str(valor)+"]/div[5]/div[2]/div/form/table/tbody/tr["+str(i)+"]/td[8]").text)
                    Año.append(año)
                    Liga.append(liga)
                except:
                    a=0
            for i in range(0,100):
                try:
                    Resultado.append(driver.find_element_by_xpath("/html/body/div[4]/div[3]/div[10]/div[2]/div["+str(valor)+"]/div[3]/div[2]/table/tbody/tr["+str(i)+"]/td[4]/a/span").text)
                    Equipo_Visitante.append(driver.find_element_by_xpath("/html/body/div[4]/div[3]/div[10]/div[2]/div["+str(valor)+"]/div[3]/div[2]/table/tbody/tr["+str(i)+"]/td[5]/a[2]").text)
                    Equipo_Local.append(driver.find_element_by_xpath("/html/body/div[4]/div[3]/div[10]/div[2]/div["+str(valor)+"]/div[3]/div[2]/table/tbody/tr["+str(i)+"]/td[3]/a[2]").text)
                    Fecha.append(driver.find_element_by_xpath("/html/body/div[4]/div[3]/div[10]/div[2]/div["+str(valor)+"]/div[3]/div[2]/table/tbody/tr["+str(i)+"]/td[2]").text)
                    Año_.append(año)
                    Jornada_.append(jornada)
                    Liga_.append(liga)
                except:
                    a=0        
            driver.quit()
            
for dic in range(0,1):
    liga="primera"
    valor=4
    for año in range (Ano1,Ano2):
        for jornada in range(1,44):
            driver=webdriver.Chrome('C:/Users/34690/Desktop/Imagenes presentacion/chromedriver.exe')
            driver.get("https://www.resultados-futbol.com/"+liga+str(año)+"/grupo1/jornada"+str(jornada))
            time.sleep(2)
            try:
                pulsar=driver.find_element_by_xpath("/html/body/div[1]/div/div/div/div[2]/div/button[2]")
                pulsar.click()
            except:
                a=0
            for i in range(0,25):
                try:
                    Posicion.append(driver.find_element_by_xpath("/html/body/div[4]/div[3]/div[10]/div[2]/div["+str(valor)+"]/div[5]/div[2]/div/form/table/tbody/tr["+str(i)+"]/th").text)
                    Equipo.append(driver.find_element_by_xpath("/html/body/div[4]/div[3]/div[10]/div[2]/div["+str(valor)+"]/div[5]/div[2]/div/form/table/tbody/tr["+str(i)+"]/td[1]").text)
                    Puntos.append(driver.find_element_by_xpath("/html/body/div[4]/div[3]/div[10]/div[2]/div["+str(valor)+"]/div[5]/div[2]/div/form/table/tbody/tr["+str(i)+"]/td[2]").text)
                    Jornada.append(driver.find_element_by_xpath("/html/body/div[4]/div[3]/div[10]/div[2]/div["+str(valor)+"]/div[5]/div[2]/div/form/table/tbody/tr["+str(i)+"]/td[3]").text)
                    Ganados.append(driver.find_element_by_xpath("/html/body/div[4]/div[3]/div[10]/div[2]/div["+str(valor)+"]/div[5]/div[2]/div/form/table/tbody/tr["+str(i)+"]/td[4]").text)
                    Empatados.append(driver.find_element_by_xpath("/html/body/div[4]/div[3]/div[10]/div[2]/div["+str(valor)+"]/div[5]/div[2]/div/form/table/tbody/tr["+str(i)+"]/td[5]").text)
                    Perdidos.append(driver.find_element_by_xpath("/html/body/div[4]/div[3]/div[10]/div[2]/div["+str(valor)+"]/div[5]/div[2]/div/form/table/tbody/tr["+str(i)+"]/td[6]").text)
                    Goles_a_favor.append(driver.find_element_by_xpath("/html/body/div[4]/div[3]/div[10]/div[2]/div["+str(valor)+"]/div[5]/div[2]/div/form/table/tbody/tr["+str(i)+"]/td[7]").text)
                    Goles_en_contra.append(driver.find_element_by_xpath("/html/body/div[4]/div[3]/div[10]/div[2]/div["+str(valor)+"]/div[5]/div[2]/div/form/table/tbody/tr["+str(i)+"]/td[8]").text)
                    Año.append(año)
                    Liga.append(liga)
                except:
                    a=0
            i=0
            for i in range(0,100):
                try:
                    Resultado.append(driver.find_element_by_xpath("/html/body/div[4]/div[3]/div[10]/div[2]/div["+str(valor)+"]/div[3]/div[2]/table/tbody/tr["+str(i)+"]/td[4]/a/span").text)
                    Equipo_Visitante.append(driver.find_element_by_xpath("/html/body/div[4]/div[3]/div[10]/div[2]/div["+str(valor)+"]/div[3]/div[2]/table/tbody/tr["+str(i)+"]/td[5]/a[2]").text)
                    Equipo_Local.append(driver.find_element_by_xpath("/html/body/div[4]/div[3]/div[10]/div[2]/div["+str(valor)+"]/div[3]/div[2]/table/tbody/tr["+str(i)+"]/td[3]/a[2]").text)
                    Fecha.append(driver.find_element_by_xpath("/html/body/div[4]/div[3]/div[10]/div[2]/div["+str(valor)+"]/div[3]/div[2]/table/tbody/tr["+str(i)+"]/td[2]").text)
                    Año_.append(año)
                    Jornada_.append(jornada)
                    Liga_.append(liga)
                except:
                    a=0          
            driver.quit()
            
Data_Tabla=pd.DataFrame([Posicion,Equipo,Puntos,Año,Jornada,Ganados,Empatados,Perdidos,Goles_a_favor,Goles_en_contra,Liga]).transpose()
Data_Tabla.columns=['Posicion','Equipo','Puntos','Año','Jornada','Ganados','Empatados','Perdidos','Goles_a_favor','Goles_en_contra','Liga']
Data_Tabla.to_excel("DDBB_Tabla_España_"+str(Ano1)+"_"+str(Ano2)+".xlsx") 

Data=pd.DataFrame([Equipo_Local,Equipo_Visitante,Resultado,Año_,Jornada_,Fecha,Liga_]).transpose()
Data.columns=['Equipo_Local','Equipo_Visitante','Resultado','Año','Jornada','Fecha','Liga']
Data.to_excel("DDBB_Resultados_España_"+str(Ano1)+"_"+str(Ano2)+".xlsx")  
  

In [ ]:
#Inglaterra
Posicion=[]
Equipo=[]
Puntos=[]
Jornada=[]
Ganados=[]
Empatados=[]
Perdidos=[]
Goles_a_favor=[]
Goles_en_contra=[]
Año=[]
Liga=[]

Equipo_Local=[]
Equipo_Visitante=[]
Resultado=[]
Año_=[]
Jornada_=[]
Fecha=[]
Liga_=[]
ano1=1990
ano2=2020
Dict_liga=['prime','segunda']
for dic in range(0,1):
    liga="premier"
    valor=5
    for año in range (ano1,ano2):
        for jornada in range(1,40):
            driver=webdriver.Chrome('C:/Users/34690/Desktop/Imagenes presentacion/chromedriver.exe')
            driver.get("https://www.resultados-futbol.com/"+liga+str(año)+"/grupo1/jornada"+str(jornada))
            time.sleep(2)
            try:
                pulsar=driver.find_element_by_xpath("/html/body/div[1]/div/div/div/div[2]/div/button[2]")
                pulsar.click()
            except:
                a=0
            for i in range(0,25):
                try:
                    Posicion.append(driver.find_element_by_xpath("/html/body/div[4]/div[3]/div[10]/div[2]/div["+str(valor)+"]/div[5]/div[2]/div/form/table/tbody/tr["+str(i)+"]/th").text)
                    Equipo.append(driver.find_element_by_xpath("/html/body/div[4]/div[3]/div[10]/div[2]/div["+str(valor)+"]/div[5]/div[2]/div/form/table/tbody/tr["+str(i)+"]/td[1]").text)
                    Puntos.append(driver.find_element_by_xpath("/html/body/div[4]/div[3]/div[10]/div[2]/div["+str(valor)+"]/div[5]/div[2]/div/form/table/tbody/tr["+str(i)+"]/td[2]").text)
                    Jornada.append(driver.find_element_by_xpath("/html/body/div[4]/div[3]/div[10]/div[2]/div["+str(valor)+"]/div[5]/div[2]/div/form/table/tbody/tr["+str(i)+"]/td[3]").text)
                    Ganados.append(driver.find_element_by_xpath("/html/body/div[4]/div[3]/div[10]/div[2]/div["+str(valor)+"]/div[5]/div[2]/div/form/table/tbody/tr["+str(i)+"]/td[4]").text)
                    Empatados.append(driver.find_element_by_xpath("/html/body/div[4]/div[3]/div[10]/div[2]/div["+str(valor)+"]/div[5]/div[2]/div/form/table/tbody/tr["+str(i)+"]/td[5]").text)
                    Perdidos.append(driver.find_element_by_xpath("/html/body/div[4]/div[3]/div[10]/div[2]/div["+str(valor)+"]/div[5]/div[2]/div/form/table/tbody/tr["+str(i)+"]/td[6]").text)
                    Goles_a_favor.append(driver.find_element_by_xpath("/html/body/div[4]/div[3]/div[10]/div[2]/div["+str(valor)+"]/div[5]/div[2]/div/form/table/tbody/tr["+str(i)+"]/td[7]").text)
                    Goles_en_contra.append(driver.find_element_by_xpath("/html/body/div[4]/div[3]/div[10]/div[2]/div["+str(valor)+"]/div[5]/div[2]/div/form/table/tbody/tr["+str(i)+"]/td[8]").text)
                    Año.append(año)
                    Liga.append(liga)
                except:
                    a=0
            for i in range(0,100):
                try:
                    Resultado.append(driver.find_element_by_xpath("/html/body/div[4]/div[3]/div[10]/div[2]/div["+str(valor)+"]/div[3]/div[2]/table/tbody/tr["+str(i)+"]/td[4]/a/span").text)
                    Equipo_Visitante.append(driver.find_element_by_xpath("/html/body/div[4]/div[3]/div[10]/div[2]/div["+str(valor)+"]/div[3]/div[2]/table/tbody/tr["+str(i)+"]/td[5]/a[2]").text)
                    Equipo_Local.append(driver.find_element_by_xpath("/html/body/div[4]/div[3]/div[10]/div[2]/div["+str(valor)+"]/div[3]/div[2]/table/tbody/tr["+str(i)+"]/td[3]/a[2]").text)
                    Fecha.append(driver.find_element_by_xpath("/html/body/div[4]/div[3]/div[10]/div[2]/div["+str(valor)+"]/div[3]/div[2]/table/tbody/tr["+str(i)+"]/td[2]").text)
                    Año_.append(año)
                    Jornada_.append(jornada)
                    Liga_.append(liga)
                except:
                    a=0        
            driver.quit()
            
for dic in range(0,1):
    liga="championship"
    valor=4
    for año in range (ano1,ano2):
        for jornada in range(1,40):
            driver=webdriver.Chrome('C:/Users/34690/Desktop/Imagenes presentacion/chromedriver.exe')
            driver.get("https://www.resultados-futbol.com/"+liga+str(año)+"/grupo1/jornada"+str(jornada))
            time.sleep(2)
            try:
                pulsar=driver.find_element_by_xpath("/html/body/div[1]/div/div/div/div[2]/div/button[2]")
                pulsar.click()
            except:
                a=0
            for i in range(0,25):
                try:
                    Posicion.append(driver.find_element_by_xpath("/html/body/div[4]/div[3]/div[10]/div[2]/div["+str(valor)+"]/div[5]/div[2]/div/form/table/tbody/tr["+str(i)+"]/th").text)
                    Equipo.append(driver.find_element_by_xpath("/html/body/div[4]/div[3]/div[10]/div[2]/div["+str(valor)+"]/div[5]/div[2]/div/form/table/tbody/tr["+str(i)+"]/td[1]").text)
                    Puntos.append(driver.find_element_by_xpath("/html/body/div[4]/div[3]/div[10]/div[2]/div["+str(valor)+"]/div[5]/div[2]/div/form/table/tbody/tr["+str(i)+"]/td[2]").text)
                    Jornada.append(driver.find_element_by_xpath("/html/body/div[4]/div[3]/div[10]/div[2]/div["+str(valor)+"]/div[5]/div[2]/div/form/table/tbody/tr["+str(i)+"]/td[3]").text)
                    Ganados.append(driver.find_element_by_xpath("/html/body/div[4]/div[3]/div[10]/div[2]/div["+str(valor)+"]/div[5]/div[2]/div/form/table/tbody/tr["+str(i)+"]/td[4]").text)
                    Empatados.append(driver.find_element_by_xpath("/html/body/div[4]/div[3]/div[10]/div[2]/div["+str(valor)+"]/div[5]/div[2]/div/form/table/tbody/tr["+str(i)+"]/td[5]").text)
                    Perdidos.append(driver.find_element_by_xpath("/html/body/div[4]/div[3]/div[10]/div[2]/div["+str(valor)+"]/div[5]/div[2]/div/form/table/tbody/tr["+str(i)+"]/td[6]").text)
                    Goles_a_favor.append(driver.find_element_by_xpath("/html/body/div[4]/div[3]/div[10]/div[2]/div["+str(valor)+"]/div[5]/div[2]/div/form/table/tbody/tr["+str(i)+"]/td[7]").text)
                    Goles_en_contra.append(driver.find_element_by_xpath("/html/body/div[4]/div[3]/div[10]/div[2]/div["+str(valor)+"]/div[5]/div[2]/div/form/table/tbody/tr["+str(i)+"]/td[7]").text)
                    Año.append(año)
                    Liga.append(liga)
                except:
                    a=0
            i=0
            for i in range(0,100):
                try:
                    Resultado.append(driver.find_element_by_xpath("/html/body/div[4]/div[3]/div[10]/div[2]/div["+str(valor)+"]/div[3]/div[2]/table/tbody/tr["+str(i)+"]/td[4]/a/span").text)
                    Equipo_Visitante.append(driver.find_element_by_xpath("/html/body/div[4]/div[3]/div[10]/div[2]/div["+str(valor)+"]/div[3]/div[2]/table/tbody/tr["+str(i)+"]/td[5]/a[2]").text)
                    Equipo_Local.append(driver.find_element_by_xpath("/html/body/div[4]/div[3]/div[10]/div[2]/div["+str(valor)+"]/div[3]/div[2]/table/tbody/tr["+str(i)+"]/td[3]/a[2]").text)
                    Fecha.append(driver.find_element_by_xpath("/html/body/div[4]/div[3]/div[10]/div[2]/div["+str(valor)+"]/div[3]/div[2]/table/tbody/tr["+str(i)+"]/td[2]").text)
                    Año_.append(año)
                    Jornada_.append(jornada)
                    Liga_.append(liga)
                except:
                    a=0          
            driver.quit()
            
Data_Tabla=pd.DataFrame([Posicion,Equipo,Puntos,Año,Jornada,Ganados,Empatados,Perdidos,Goles_a_favor,Goles_en_contra,Liga]).transpose()
Data_Tabla.columns=['Posicion','Equipo','Puntos','Año','Jornada','Ganados','Empatados','Perdidos','Goles_a_favor','Goles_en_contra','Liga']
Data_Tabla.to_excel("DDBB_Tabla_Inglaterra_"+str(ano1)+"_"+str(ano2)+".xlsx")  

Data=pd.DataFrame([Equipo_Local,Equipo_Visitante,Resultado,Año_,Jornada_,Fecha,Liga_]).transpose()
Data.columns=['Equipo_Local','Equipo_Visitante','Resultado','Año','Jornada','Fecha','Liga']
Data.to_excel("DDBB_Resultados_Inglaterra_"+str(ano1)+"_"+str(ano2)+".xlsx")

In [ ]:
ch

NameError: ignored